In [471]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
from sklearn.metrics import confusion_matrix, f1_score, precision_score,\
recall_score, confusion_matrix, classification_report, accuracy_score 
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, Normalizer

In [472]:
def __GetLIWC(file:str): 
	liwc = pd.read_csv(file)
	liwc = liwc.rename(columns = {liwc.columns[2]:'user_id'})
	liwcUser = liwc.groupby('user_id').mean().reset_index()
	liwcUser = liwcUser.drop(['Source (A)', 'Source (D)'], axis=1)
	return liwcUser

def mergeFea(features, liwc, empath): 
	features = pd.read_csv(features)

	#merge features
	liwcUser = __GetLIWC(liwc)
	liwcUser2 = liwcUser.iloc[:,1::]
	liwcUser2.columns = [str(col) + '_liwc' for col in liwcUser2.columns]
	liwcUser2['user_id'] = liwcUser.user_id

	empath = pd.read_csv(empath)
	empath2 = empath.iloc[:,1::]
	empath2.columns = [str(col) + '_empath' for col in empath2.columns]
	empath2['user_id'] = empath.user_id

	allfea = pd.merge(features, liwcUser2, on = 'user_id', how = 'right')
	allfea = pd.merge(allfea, empath2, on = 'user_id', how = 'right')
	return allfea

def preprocess2(sent):
    #remove punctustion
    sent = re.sub(r'[^\w\s]','',str(sent))
    words = sent.split()
    new_words = []
    for w in words:      
        new_words.append(w.lower())
        
    return ' '.join(new_words)


class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key].values.reshape(-1,1)
    
class ItemSelectorText(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [317]:
obj = ItemSelector('motivations')
obj.transform(X_train)

array([[4],
       [0],
       [4],
       [1],
       [0],
       [3],
       [3],
       [5],
       [5],
       [0],
       [3],
       [5],
       [2],
       [5],
       [1],
       [1],
       [4],
       [5],
       [2],
       [0],
       [5],
       [5],
       [5],
       [5],
       [4],
       [2],
       [5],
       [5],
       [3],
       [5],
       [0],
       [2],
       [3],
       [5],
       [5],
       [5],
       [3],
       [5],
       [1],
       [4],
       [5],
       [2],
       [4],
       [4],
       [4],
       [0],
       [5],
       [1],
       [4],
       [4],
       [5],
       [3],
       [5],
       [2],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [2],
       [2],
       [3],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [4],
       [2],
       [5],
       [5],
       [4],
       [2],
       [4],
       [5],
       [3],
       [2],
       [3],
       [4],
    

In [318]:
path = '/home/lucia/phd_work/shareTask/'

# #merge features
# features = path + 'suicideDetection/features/FreqSentiMotiTopiFea.csv'
# liwc = path + 'suicideDetection/features/liwcSW.csv'
# empath = path + 'suicideDetection/features/empathSW.csv'
# allfea = mergeFea(features, liwc, empath)

# #select features and split train test
# X = allfea.iloc[:, 3:146]
# y = allfea.raw_label

In [319]:
X = pd.read_csv(path + '/data/clpsych19_training_data/Btrain_NoNoise_SW.csv')
y = pd.read_csv(path + '/data/clpsych19_training_data/crowd_train.csv')


In [320]:
#concatenate text according to user id
X1 = X[['user_id','post_body']]
conTex = X1.groupby(['user_id'],as_index=False).agg(lambda x : x.sum() if str(x) else ' '.join(x))


In [321]:
conTex['post_body'] = conTex['post_body'].apply(lambda x: preprocess2(x))

In [440]:
allFea = pd.read_csv(path + '/suicideDetection/features/FreqSentiMotiTopiFea.csv')
liwc = pd.read_csv(path + '/suicideDetection/features/liwcSW.csv')
liwc = liwc.iloc[:,np.r_[2, 8:liwc.shape[1]]]
liwc = liwc.rename(columns = {liwc.columns[0]:'user_id'})
liwcUser = liwc.groupby('user_id').mean().reset_index()
liwcUser.columns = [str(col) + '_liwc' for col in liwcUser.columns]
empath = pd.read_csv(path + '/suicideDetection/features/empathSW.csv')
empath.columns = [str(col) + '_empath' for col in empath.columns]
allFea = pd.merge(allFea, liwcUser, left_on ='user_id', right_on = 'user_id_liwc', how = 'left')
allFea = pd.merge(allFea, empath, left_on ='user_id', right_on = 'user_id_empath', how = 'left')

# # # #liwc
# # # allFea


In [441]:
#fea = pd.merge(y, conTex, on = 'user_id')
allFea2  = pd.merge(conTex, allFea, on = 'user_id')
#X = fea['post_body']
y = allFea2.raw_label

In [442]:
X_train, X_test, y_train, y_test = train_test_split(allFea2, y, test_size=0.30, random_state=35)

In [443]:
print(X_train.shape)
print(y_train.shape)

(347, 158)
(347,)


In [444]:
X_train.columns

Index(['user_id', 'post_body', 'Unnamed: 0', 'raw_label', 'postingFrequency',
       'postingInterval', 'generalMoreFreq', 'generalWordCount',
       'healthPostingFrequency', 'healthPostingInterval',
       ...
       'attractive_empath', 'banking_empath', 'money_empath', 'body_empath',
       'health_empath', 'injury_empath', 'medical_emergency_empath',
       'death_empath', 'friends_empath', 'help_empath'],
      dtype='object', length=158)

In [481]:
get_liwc_data = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('liwc')]], validate=False)
get_empath_data = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('empath')]], validate=False)

pipe1 = Pipeline([
    
    ('feats', FeatureUnion([
        ('text', Pipeline([
            ('selector', ItemSelectorText(key='post_body')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
             ])),
        ('liwc_features', Pipeline([
                ('selector', get_liwc_data) ])),
#         ('empath_features', Pipeline([
#                 ('selector', get_empath_data) ])),

        ('selector1', ItemSelector(key='motivations')),
        ('selector2', ItemSelector(key='healthPostingFrequency')),
        ('selector3', ItemSelector(key='healthPostingInterval')),
        ('selector4', ItemSelector(key='healthMoreFreq')),
        ('selector5', ItemSelector(key='healthWordCount')),
        ('selector6', ItemSelector(key='mentionMethods')),
        ('selector7', ItemSelector(key='SWFrequency')),
        ('selector8', ItemSelector(key='SWPostingInterval')),
        ('selector9', ItemSelector(key='SWFreq')),
        ('selector10', ItemSelector(key='suicide_body')),
        ('selector11', ItemSelector(key='hopeless_body')),
        ('selector12', ItemSelector(key='self_senti')),
        ('selector13', ItemSelector(key='mclust')),
             ])),
    
    ('clf', Pipeline([
     ('scale', StandardScaler(with_mean=False)),
     #('feature_selection', SelectFromModel(LinearSVC(penalty="l2"))),
    ('svm', svm.SVC()),
         ])),
])

In [479]:
pipe1.fit(X_train, y_train)

/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('feats', FeatureUnion(n_jobs=None,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('selector', ItemSelectorText(key='post_body')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='c... max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]))])

In [448]:
pipe1.steps

[('feats', FeatureUnion(n_jobs=None,
         transformer_list=[('text', Pipeline(memory=None,
       steps=[('selector', ItemSelectorText(key='post_body')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_f...body')), ('selector12', ItemSelector(key='self_senti')), ('selector13', ItemSelector(key='mclust'))],
         transformer_weights=None)), ('clf', Pipeline(memory=None,
       steps=[('logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]))]

In [480]:
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           a       0.38      0.17      0.24        35
           b       0.00      0.00      0.00        18
           c       0.00      0.00      0.00        32
           d       0.44      0.91      0.59        64

   micro avg       0.43      0.43      0.43       149
   macro avg       0.20      0.27      0.21       149
weighted avg       0.28      0.43      0.31       149



/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


check parameter names

In [307]:
pipe1.named_steps['logit'].get_params()

KeyError: 'logit'

In [308]:
pipe1.get_params()

{'clf': Pipeline(memory=None,
      steps=[('logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l2', random_state=None, solver='warn',
           tol=0.0001, verbose=0, warm_start=False))]),
 'clf__logit': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l2', random_state=None, solver='warn',
           tol=0.0001, verbose=0, warm_start=False),
 'clf__logit__C': 1.0,
 'clf__logit__class_weight': None,
 'clf__logit__dual': False,
 'clf__logit__fit_intercept': True,
 'clf__logit__intercept_scaling': 1,
 'clf__logit__max_iter': 100,
 'clf__logit__multi_class': 'warn',
 'clf__logit__n_jobs': None,
 'clf__logit__penalty': 'l2',
 'clf__logit__random_state': None,
 'clf__logit__solver': 'warn',
 'clf__logit__tol': 0.0001,
 'clf__logit__v

In [309]:
pipe1.set_params(cv__min_df=6, 
                 cv__lowercase=False).fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

ValueError: Invalid parameter cv for estimator Pipeline(memory=None,
     steps=[('feats', FeatureUnion(n_jobs=None,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('selector', ItemSelectorText(key='post_body')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='c...alty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [175]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter

class PosTagMatrix(BaseEstimator, TransformerMixin):
    #normalise = True - devide all values by a total number of tags in the sentence
    #tokenizer - take a custom tokenizer function
    def __init__(self, tokenizer=lambda x: x.split(), normalize=True):
        self.tokenizer=tokenizer
        self.normalize=normalize

    #helper function to tokenize and count parts of speech
    def pos_func(self, sentence):
        return Counter(tag for word,tag in nltk.pos_tag(self.tokenizer(sentence)))

    # fit() doesn't do anything, this is a transformer class
    def fit(self, X, y = None):
        return self

    #all the work is done here
    def transform(self, X):
        X_tagged = X.apply(self.pos_func).apply(pd.Series).fillna(0)
        X_tagged['n_tokens'] = X_tagged.apply(sum, axis=1)
        if self.normalize:
            X_tagged = X_tagged.divide(X_tagged['n_tokens'], axis=0)

        return X_tagged

In [197]:
obj = PosTagMatrix()
tranTag = obj.transform(X_train)
tranTag.columns

Index(['CD', 'NNS', 'RB', 'NN', 'VBD', 'CC', 'VBG', 'IN', 'DT', 'PRP$', 'VBP',
       'JJ', 'PRP', 'VBN', 'VBZ', 'TO', 'VB', 'MD', 'RP', 'WP', 'WRB', 'JJR',
       'EX', 'RBR', 'JJS', 'WDT', 'PDT', 'RBS', 'NNP', 'FW', '$', 'UH', 'WP$',
       'n_tokens'],
      dtype='object')

In [ ]:
testTag = obj.transform(X_test)
tran.columns

In [ ]:
test_x2 = test_x.loc[:, test_x.columns.isin(tranTag)]

In [172]:
from sklearn.pipeline import FeatureUnion

pipe2 = Pipeline([
    ('u1', FeatureUnion([
        ('tfdif_features', Pipeline([
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])),
        ('pos_features', Pipeline([
            ('pos', PosTagMatrix(tokenizer=nltk.word_tokenize) ),
        ])),
    ])),
    ('logit', LogisticRegression()),

])

In [194]:
print(X_train.shape)
print(X_test.shape)

(347,)
(149,)


In [186]:
pipe2.fit(X_train, y_train)
pred = pipe2.predict(X_test)
#print(classification_report(y_test, pred))

/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


ValueError: X has 8634 features per sample; expecting 8635